# Train models for visual proprioception

Train a regression model for visual proprioception. 

The encoding used must be specified in an experiment of the type "visual_proprioception". Running this notebook will train and save this model.

In [2]:
import sys
sys.path.append("..")
from settings import Config

import pathlib
from pprint import pprint
import matplotlib.pyplot as plt

import numpy as np
import torch
import torch.nn as nn
#import torch.nn.functional as F
import torch.optim as optim
from torch.utils.data import DataLoader, TensorDataset

torch.manual_seed(1)

# from behavior_cloning.demo_to_trainingdata import BCDemonstration
from sensorprocessing import sp_conv_vae, sp_propriotuned_cnn
# from robot.al5d_position_controller import RobotPosition

from visual_proprioception.visproprio_helper import load_demonstrations_as_proprioception_training
from visual_proprioception.visproprio_models import VisProprio_SimpleMLPRegression

# Move data to GPU (if available)
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

Loading pointer config file: C:\Users\lboloni\.config\BerryPicker\mainsettings.yaml
Loading machine-specific config file: G:\My Drive\LotziStudy\Code\PackageTracking\BerryPicker\settings\settings-LotziYoga.yaml


## Create the sensor processing units

Load the visual_proprioception experiment/runs and based on what is written in there, load the exp/runs specific to the sensorprocessing package, and create the sp objects.

In [3]:
experiment = "visual_proprioception"
# run = "vp_mpl_conv_vae_1"
run = "vp_mpl_propriotuned_cnn_1"
exp = Config().get_experiment(experiment, run)
pprint(exp)

if exp["sensor_processing"] == "ConvVaeSensorProcessing":
    spexp = Config().get_experiment(
        exp['sp_experiment'], exp['sp_run'])
    sp = sp_conv_vae.ConvVaeSensorProcessing(exp)
elif exp['sensor_processing']=="VGG19ProprioTunedSensorProcessing":
    spexp = Config().get_experiment(exp['sp_experiment'], exp['sp_run'])
    sp = sp_propriotuned_cnn.VGG19ProprioTunedSensorProcessing(spexp, device)
else:
    raise Exception('Unknown sensor processing {exp["sensor_processing"]}')

Note: no system dependent config file G:\My Drive\LotziStudy\Code\PackageTracking\BerryPicker\settings\experiment-config\LotziYoga\visual_proprioception\vp_mpl_propriotuned_cnn_1_sysdep.yaml,
 that is ok, proceeding.
Configuration for experiment: visual_proprioception/vp_mpl_propriotuned_cnn_1 successfully loaded
{'data_dir': WindowsPath('c:/Users/lboloni/Documents/Code/_TempData/BerryPicker-experiments/visual_proprioception/vp_mpl_propriotuned_cnn_1'),
 'encoding_size': 256,
 'epochs': 30,
 'group_name': 'visual_proprioception',
 'output_size': 6,
 'proprioception_input_file': 'train_inputs.pt',
 'proprioception_mlp_model_file': 'proprioception_mlp.pth',
 'proprioception_target_file': 'train_targets.pt',
 'proprioception_test_input_file': 'test_inputs.pt',
 'proprioception_test_target_file': 'test_targets.pt',
 'proprioception_testing_task': 'proprio_regressor_validation',
 'proprioception_training_task': 'proprio_regressor_training',
 'regressor_hidden_size_1': 64,
 'regressor_hidden

c:\Users\lboloni\Documents\Code\_VirtualEnvironments\Robot\Robot-venv\Lib\site-packages\torchvision\models\_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
c:\Users\lboloni\Documents\Code\_VirtualEnvironments\Robot\Robot-venv\Lib\site-packages\torchvision\models\_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=VGG19_Weights.IMAGENET1K_V1`. You can also use `weights=VGG19_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)
c:\Users\lboloni\Documents\Code\_Checkouts\BerryPicker\src\visual_proprioception\..\sensorprocessing\sp_propriotuned_cnn.py:84: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possibl

In [4]:
# Create the regression model 

model = VisProprio_SimpleMLPRegression(exp)
# criterion = nn.MSELoss()
# Experiment: would this be better???
criterion = nn.L1Loss()
optimizer = optim.Adam(model.parameters(), lr=0.001)

In [5]:
# Load the training data

task = exp["proprioception_training_task"]
proprioception_input_file = pathlib.Path(
    exp["data_dir"], exp["proprioception_input_file"])
proprioception_target_file = pathlib.Path(
    exp["data_dir"], exp["proprioception_target_file"])
tr = load_demonstrations_as_proprioception_training(
    sp, task, proprioception_input_file, proprioception_target_file)
inputs_training = tr["inputs_training"]
targets_training = tr["targets_training"]
inputs_validation = tr["inputs_validation"]
targets_validation = tr["targets_validation"]

# Create DataLoaders for batching
batch_size = 32
train_dataset = TensorDataset(inputs_training, targets_training)
test_dataset = TensorDataset(inputs_validation, targets_validation)

train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True)
test_loader = DataLoader(test_dataset, batch_size=batch_size, shuffle=False)

Cameras found: ['dev2']
There are 547 steps in this demonstration
This demonstration was recorded by the following cameras: ['dev2']
{'actiontype': 'rc-position-target',
 'camera': 'dev2',
 'cameras': ['dev2'],
 'maxsteps': 547,
 'sensorprocessor': <sensorprocessing.sp_propriotuned_cnn.VGG19ProprioTunedSensorProcessing object at 0x0000025535852850>,
 'source_dir': WindowsPath('C:/Users/lboloni/Documents/Code/_TempData/BerryPicker-demos/demos/proprio_regressor_training/2024_12_26__16_45_02'),
 'trim_from': 1,
 'trim_to': 547}
sensor readings shape torch.Size([1, 3, 256, 256])
sensor readings shape torch.Size([1, 3, 256, 256])
sensor readings shape torch.Size([1, 3, 256, 256])
sensor readings shape torch.Size([1, 3, 256, 256])
sensor readings shape torch.Size([1, 3, 256, 256])
sensor readings shape torch.Size([1, 3, 256, 256])
sensor readings shape torch.Size([1, 3, 256, 256])
sensor readings shape torch.Size([1, 3, 256, 256])
sensor readings shape torch.Size([1, 3, 256, 256])
sensor rea

In [6]:
def train_and_save_proprioception_model(modelfile, epochs=20):
    """Trains and saves the proprioception model
    FIXME: must have parameters etc to investigate alternative models. 
    """

    # Training loop
    num_epochs = epochs
    for epoch in range(num_epochs):
        model.train()
        total_loss = 0
        for batch_X, batch_y in train_loader:
            # Forward pass
            predictions = model(batch_X)
            loss = criterion(predictions, batch_y)
            
            # Backward pass and optimization
            optimizer.zero_grad()
            loss.backward()
            optimizer.step()
            
            total_loss += loss.item()
        
        if (epoch + 1) % 10 == 0:
            print(f'Epoch [{epoch+1}/{num_epochs}], Loss: {total_loss / len(train_loader):.4f}')

    # Evaluate the model
    model.eval()
    test_loss = 0
    with torch.no_grad():
        for batch_X, batch_y in test_loader:
            predictions = model(batch_X)
            loss = criterion(predictions, batch_y)
            test_loss += loss.item()

    test_loss /= len(test_loader)
    print(f'Test Loss: {test_loss:.4f}')
    torch.save(model.state_dict(), modelfile)

In [7]:
# modelfile = pathlib.Path(Config()["explorations"]["proprioception_mlp_model_file"])
modelfile = pathlib.Path(exp["data_dir"], 
                         exp["proprioception_mlp_model_file"])
epochs = exp["epochs"]
if modelfile.exists():
    model.load_state_dict(torch.load(modelfile))
else:
    train_and_save_proprioception_model(modelfile, epochs=epochs)

RuntimeError: mat1 and mat2 shapes cannot be multiplied (32x128 and 256x64)